## plot the result to inspect

In [ ]:
import dash
from dash import html, dcc
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import geopandas as gpd
import pandas as pd
import plotly.express as px
import json
import numpy as np
import os
from shapely.geometry import Point
from shapely import wkt
# Process the 'reeds_ba_list' column to expand the sets into individual rows, keeping 'state' intact
from ast import literal_eval
# Get the current directory where your script is running
current_directory = os.getcwd()

# Construct the path to your resources folder dynamically
resources_path = os.path.join(current_directory, 'resources')

# Now, build the full path to your CSV files
polygons_csv_path = os.path.join(resources_path, 'US_CAN_MEX_PCA_polygons.csv')
state_to_ba_csv_path = os.path.join(resources_path, 'state_to_ba_mapping.csv')
# Finally, use pandas to read the CSV files
polygons_df = pd.read_csv(polygons_csv_path)
state_to_ba_df = pd.read_csv(state_to_ba_csv_path)
# Filter polygons
polygons_df = polygons_df[polygons_df['rb'].isin([f'p{i}' for i in range(1, 135)])]

# Since the instructions for processing the state mapping are a bit unclear,
# Convert the string representation of sets in 'reeds_ba_list' to actual sets
state_to_ba_df['reeds_ba_list'] = state_to_ba_df['reeds_ba_list'].apply(literal_eval)

# Explode the sets into separate rows
exploded_df = state_to_ba_df.explode('reeds_ba_list')[['state','reeds_ba_list']]
polygons_df_filtered = polygons_df[polygons_df['rb'].isin([f'p{i}' for i in range(1, 135)])]
polygons_gdf = gpd.GeoDataFrame(polygons_df_filtered, geometry=gpd.GeoSeries.from_wkt(polygons_df_filtered['WKT']))

# Step 2: Merge exploded_df with polygons_gdf on the 'reeds_ba_list' and 'rb' columns
merged_gdf = polygons_gdf.merge(exploded_df, left_on='rb', right_on='reeds_ba_list')
merged_gdf['country']='USA'
merged_gdf=merged_gdf[['rb','state','country','geometry']]

# Country - Group by 'country' and dissolve to merge geometries
gdf_country = merged_gdf.dissolve(by='country').reset_index()

# State - Group by 'state' and dissolve to merge geometries
gdf_state = merged_gdf.dissolve(by='state').reset_index()

# Subregion (rb) - No need to group since each 'rb' is already unique
gdf_subregion = merged_gdf[['rb', 'geometry']].drop_duplicates()


# Convert each GeoDataFrame to GeoJSON for use in Plotly
geojson_country = json.loads(gdf_country.to_json())
geojson_state = json.loads(gdf_state.to_json())
geojson_subregion = json.loads(gdf_subregion.to_json())
import json
current_directory= os.path.join(current_directory, 'web_page_data')
# Define file paths for saving
geojson_country_path = os.path.join(current_directory, 'geojson_country.json')
geojson_state_path = os.path.join(current_directory, 'geojson_state.json')
geojson_subregion_path = os.path.join(current_directory, 'geojson_subregion.json')
# Define file paths for saving
gdf_country_path = os.path.join(current_directory, 'gdf_country.gpkg')
gdf_state_path = os.path.join(current_directory, 'gdf_state.gpkg')
gdf_subregion_path = os.path.join(current_directory, 'gdf_subregion.gpkg')

# Save the GeoDataFrames
gdf_country.to_file(gdf_country_path, driver='GPKG')
gdf_state.to_file(gdf_state_path, driver='GPKG')
gdf_subregion.to_file(gdf_subregion_path, driver='GPKG')

# Function to save GeoJSON to a file
def save_geojson(data, file_path):
    with open(file_path, 'w') as f:
        json.dump(data, f)

# Save the GeoJSON data
save_geojson(geojson_country, geojson_country_path)
save_geojson(geojson_state, geojson_state_path)
save_geojson(geojson_subregion, geojson_subregion_path)


In [ ]:
# Function to read GeoJSON from a file
def read_geojson(file_path):
    with open(file_path) as f:
        return json.load(f)

# Read the GeoJSON data
geojson_country_read = read_geojson(geojson_country_path)
geojson_state_read = read_geojson(geojson_state_path)
geojson_subregion_read = read_geojson(geojson_subregion_path)


In [ ]:
print(geojson_country == geojson_country_read)
print(geojson_state == geojson_state_read)
print(geojson_subregion == geojson_subregion_read)


In [ ]:
# Read the GeoDataFrames
gdf_country_read = gpd.read_file(gdf_country_path)
gdf_state_read = gpd.read_file(gdf_state_path)
gdf_subregion_read = gpd.read_file(gdf_subregion_path)


In [ ]:
print(gdf_country.shape == gdf_country_read.shape)
print(gdf_state.shape == gdf_state_read.shape)
print(gdf_subregion.shape == gdf_subregion_read.shape)

# Sample a row and compare geometries (example using 'gdf_country')
print(gdf_country.geometry.iloc[0].equals(gdf_country_read.geometry.iloc[0]))


In [3]:
# Assuming you have a GeoJSON file and a corresponding GeoDataFrame
import geopandas as gpd
import json
import os
current_directory = os.getcwd()
data_path = os.path.join(current_directory, 'web_page_data')

geojson_country_path = os.path.join(data_path, 'geojson_country.json')
geojson_state_path = os.path.join(data_path, 'geojson_state.json')
geojson_subregion_path = os.path.join(data_path, 'geojson_subregion.json')

# Define file paths for saving
gdf_country_path = os.path.join(data_path, 'gdf_country.gpkg')
gdf_state_path = os.path.join(data_path, 'gdf_state.gpkg')
gdf_subregion_path = os.path.join(data_path, 'gdf_subregion.gpkg')

# Load GeoJSON
with open(geojson_country_path) as f:
    geojson_country = json.load(f)
with open(geojson_state_path) as f:
    geojson_state = json.load(f)
with open(geojson_subregion_path) as f:
    geojson_subregion = json.load(f)

# Check the first GeoJSON feature properties
print("GeoJSON feature example:", geojson_subregion['features'][0]['properties'])

# Load GeoDataFrame and check its first row
gdf_subregion = gpd.read_file(gdf_subregion_path)
print("GeoDataFrame example:", gdf_subregion.iloc[0])

# Ensure the 'locations' parameter in Plotly Express matches these identifiers


GeoJSON feature example: {'rb': 'p1'}
GeoDataFrame example: rb                                                         p1
geometry    MULTIPOLYGON (((-122.960715524824 47.144493305...
Name: 0, dtype: object


create monthly base base line chart data

In [131]:

import pandas as pd
import os
from datetime import datetime

# Define the directory where the CSV files are located
directory = '/Volumes/T7/prediction/rcp45hotter'

# Create a list of all 'rb' codes and years
rb_codes = [f'p{i}' for i in range(1, 135)]



# Function to read and process files for a given rb code
def process_rb_files(rb_code):
    rb_df_list = []
    # Loop through all years for the given rb_code
    for year in years:
        # Construct file name
        folder_path = os.path.join(directory, str(year))
        file_name = f"{rb_code}_{year}_mlp_output.csv"
        file_path = os.path.join(folder_path, file_name)
        # Check if file exists to avoid FileNotFoundError
        if os.path.isfile(file_path):
            # Read the CSV file
            temp_df = pd.read_csv(file_path)
            temp_df=temp_df[['Time_UTC','Load']]
            # Convert 'Time_UTC' to datetime and extract 'Year' and 'Month'
            temp_df['Time_UTC'] = pd.to_datetime(temp_df['Time_UTC'])
            temp_df['Year'] = temp_df['Time_UTC'].dt.year
            temp_df['Month'] = temp_df['Time_UTC'].dt.month
            # Aggregate Load by 'Year' and 'Month'
            aggregated_df = temp_df.groupby(['Year', 'Month'])['Load'].sum().reset_index()
            # Add the aggregated data to the list
            rb_df_list.append(aggregated_df)
    
    # Vertically concatenate all yearly data for the rb code
    concatenated_df = pd.concat(rb_df_list, ignore_index=True)
    return concatenated_df

years = list(range(2020, 2100))
months = list(range(1, 13))

# Create a DataFrame with all combinations of 'Year' and 'Month'
year_month_df = pd.DataFrame([(y, m) for y in years for m in months], columns=['Year', 'Month'])

# Placeholder to simulate merging data for multiple rb codes
final_df = year_month_df.copy()
for rb_code in rb_codes: 
    rb_df = process_rb_files(rb_code)
    final_df = final_df.merge(rb_df, on=['Year', 'Month'], how='left').rename(columns={'Load': rb_code})
# Placeholder function to read 'state_to_ba_mapping.csv' and perform aggregation
def string_to_set(string):
    # Remove curly braces and split the string
    elements = string.replace('{', '').replace('}', '').split(',')
    # Remove any extra whitespace and single quotes from each element
    elements = [e.strip().replace("'", "") for e in elements]
    return set(elements)
def aggregate_state_data(final_df):
    # Placeholder to simulate reading the 'state_to_ba_mapping.csv' file
    # In practice, this should read the actual file
    state_to_ba_mapping = pd.read_csv('/Users/ansonkong/Downloads/demand_prediciton_with_weather/resources/state_to_ba_mapping.csv')

    # Add columns for each state by summing the relevant rb columns based on the mapping
    for index, row in state_to_ba_mapping.iterrows():
        rb_list = string_to_set(row['reeds_ba_list'])
        state_column = row['state']
        # Sum the columns specified in reeds_ba_list for each state
        final_df[state_column] = final_df[list(rb_list)].sum(axis=1)

    # Calculate the total for the USA by summing all state columns
    state_columns = state_to_ba_mapping['state'].values
    final_df['USA'] = final_df[state_columns].sum(axis=1)

    return final_df

# Apply the placeholder aggregation function to the simulated final_df
final_aggregated_df = aggregate_state_data(final_df)

def aggregate_state_data(final_df):
    # Placeholder to simulate reading the 'state_to_ba_mapping.csv' file
    # In practice, this should read the actual file
    state_to_ba_mapping = pd.read_csv('/Users/ansonkong/Downloads/demand_prediciton_with_weather/resources/state_to_ba_mapping.csv')

    # Add columns for each state by summing the relevant rb columns based on the mapping
    for index, row in state_to_ba_mapping.iterrows():
        rb_list = string_to_set(row['reeds_ba_list'])
        state_column = row['state']
        # Sum the columns specified in reeds_ba_list for each state
        final_df[state_column] = final_df[list(rb_list)].sum(axis=1)

    # Calculate the total for the USA by summing all state columns
    state_columns = state_to_ba_mapping['state'].values
    final_df['USA'] = final_df[state_columns].sum(axis=1)

    return final_df

# Apply the placeholder aggregation function to the simulated final_df
final_aggregated_df = aggregate_state_data(final_df)
final_aggregated_df.to_csv('/Users/ansonkong/Downloads/demand_prediciton_with_weather/resources/mock_rcp45hotter.csv')

max monthlly data

In [132]:

import pandas as pd
import os
from datetime import datetime
import numpy as np
# Placeholder function to read 'state_to_ba_mapping.csv' and perform aggregation
def string_to_set(string):
    # Remove curly braces and split the string
    elements = string.replace('{', '').replace('}', '').split(',')
    # Remove any extra whitespace and single quotes from each element
    elements = [e.strip().replace("'", "") for e in elements]
    return set(elements)
# Define the directory where the CSV files are located
directory = '/Volumes/T7/prediction/rcp45hotter'

# Create a list of all 'rb' codes and years
rb_codes = [f'p{i}' for i in range(1, 135)]
years = list(range(2020, 2100))
months = list(range(1, 13))


def process_rb_files_daily(rb_code, years):
    rb_df_list = []
    for year in years:
        folder_path = os.path.join(directory, str(year))
        file_name = f"{rb_code}_{year}_mlp_output.csv"
        file_path = os.path.join(folder_path, file_name)
        if os.path.isfile(file_path):
            temp_df = pd.read_csv(file_path)[['Time_UTC', 'Load']]
            temp_df['Time_UTC'] = pd.to_datetime(temp_df['Time_UTC'])
            rb_df_list.append(temp_df)
    concatenated_df = pd.concat(rb_df_list, ignore_index=True)
    return concatenated_df
def aggregate_state_and_usa_daily(final_rb_df):
    # Read the state to rb mapping
    state_to_ba_mapping = pd.read_csv('/Users/ansonkong/Downloads/demand_prediciton_with_weather/resources/state_to_ba_mapping.csv')
    
    # For each state, sum the relevant 'rb' daily sums
    for index, row in state_to_ba_mapping.iterrows():
        rb_list = string_to_set(row['reeds_ba_list'])  # Convert string to list/set of rb codes
        state_column = row['state']
        # Ensure only existing columns are summed
        rb_columns = [rb for rb in rb_list if rb in final_rb_df.columns]
        # Summing for each state
        final_rb_df[state_column] = final_rb_df[rb_columns].sum(axis=1)
    
    # Summing all states to get USA total
    state_columns = state_to_ba_mapping['state'].tolist()
    final_rb_df['USA'] = final_rb_df[state_columns].sum(axis=1)
    
    return final_rb_df



final_dfs = []
for year in years:
    # Process data for the year (pseudo-code placeholders for actual processing)
    First=True
    for rb_code in rb_codes:
        temp_df = process_rb_files_daily(rb_code, [year])
        if First:
            processed_df=temp_df.copy()
            First=False
            processed_df[rb_code]=processed_df['Load']
            processed_df=processed_df.drop(['Load'], axis=1)
        else:
            # Assuming processed_df and temp_df are defined and have the appropriate columns
            processed_df = processed_df.merge(temp_df, on=['Time_UTC'], how='inner')
            processed_df[rb_code]=processed_df['Load']
            processed_df=processed_df.drop(['Load'], axis=1)
    processed_df = aggregate_state_and_usa_daily(processed_df)
    processed_df['Time_UTC'] = pd.to_datetime(processed_df['Time_UTC'])
    processed_df['Year'] = processed_df['Time_UTC'].dt.year
    processed_df['Month'] = processed_df['Time_UTC'].dt.month


    processed_df=processed_df.drop(['Time_UTC'], axis=1)
    # Group by 'weekday' and 'Year', then apply the aggregations
    # Define aggregations to apply 'max' to all columns except 'Year', 'Month', and 'weekday'
    aggregations = {col: 'max' for col in processed_df.columns if col not in ['Year', 'Month']}

    # Group by 'Year' and 'Month', then aggregate
    grouped_df = processed_df.groupby(['Year', 'Month']).agg(aggregations)



    # Reset index to turn 'Year' and 'Month' back into columns if needed
    grouped_df.reset_index(inplace=True)
    final_dfs.append(grouped_df)


    

final_df = pd.concat(final_dfs, axis=0, ignore_index=True)
final_df.to_csv('/Users/ansonkong/Downloads/demand_prediciton_with_weather/resources/max_rcp45hotter_monthlly.csv')
final_df

,Year,Month,p1,p2,p3,p4,p5,p6,p7,p8,...,Tennessee,Texas,Utah,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming,USA
0,2020,1,8615.18,4793.26,2036.83,832.01,6104.79,1661.92,187.03,167.73,...,17521.11,50522.09,3832.90,874.53,18136.50,16261.59,4982.06,9787.74,2553.96,549588.71
1,2020,2,7795.90,4050.23,1873.35,708.15,5562.91,1404.89,159.52,149.00,...,16772.85,49109.52,3657.48,830.11,18685.32,14401.63,5044.18,9335.28,2288.55,518193.27
2,2020,3,6910.41,3809.57,1670.41,609.16,4840.83,1384.91,157.33,151.61,...,19338.05,48387.42,3528.05,777.78,18536.31,12873.22,5220.70,8906.96,2171.30,534674.45
3,2020,4,6387.36,3477.70,1530.21,556.14,4461.60,1274.86,154.00,160.69,...,13608.08,46650.30,3430.94,693.44,13656.91,11894.35,4097.51,8732.95,1937.93,454568.24
4,2020,5,5677.97,3281.63,1478.82,613.83,4551.67,1384.70,182.95,178.94,...,14993.12,59076.73,3914.39,733.33,17803.74,10921.70,4493.19,10111.27,2040.36,544850.03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
955,2099,8,6046.36,3815.73,2022.19,803.33,5447.76,1627.96,295.75,246.91,...,18263.33,70386.26,4662.26,1059.87,22859.80,12636.68,5673.41,12249.70,2539.25,707364.95
956,2099,9,5935.34,3750.42,1917.41,716.66,5408.21,1590.67,265.12,234.35,...,17987.79,67879.25,4540.83,909.87,21075.54,12282.95,5365.10,11115.57,2391.83,661661.07
957,2099,10,6670.47,3491.82,1573.73,587.70,4662.56,1346.99,161.79,189.75,...,18266.75,60877.47,3630.65,821.61,20298.84,12173.40,5318.98,10604.28,2193.44,604767.22
958,2099,11,7215.22,3853.08,1669.61,620.34,4944.24,1387.86,161.88,186.72,...,14816.36,46108.47,3701.84,812.84,15356.98,13104.28,4341.57,9035.56,2172.66,491079.99


In [ ]:
# Step 1: Modify process_rb_files function for daily aggregation

# Step 2: For each state and the USA, calculate daily sums

# Step 3: Calculate statistics (mean, 5% lower quantile, 95% upper quantile)
#         Separate calculations for weekdays and weekends

# Step 4: Store results with specified column names

# Step 5: Add 'Year Record' column

# Step 6: Vertically concatenate data for all years


weekly comparsion

In [133]:

import pandas as pd
import os
from datetime import datetime
import numpy as np

# Define the directory where the CSV files are located
directory = '/Volumes/T7/prediction/rcp45hotter'

# Create a list of all 'rb' codes and years
rb_codes = [f'p{i}' for i in range(1, 135)]
years = list(range(2020, 2100))
months = list(range(1, 13))


def process_rb_files_daily(rb_code, years):
    rb_df_list = []
    for year in years:
        folder_path = os.path.join(directory, str(year))
        file_name = f"{rb_code}_{year}_mlp_output.csv"
        file_path = os.path.join(folder_path, file_name)
        if os.path.isfile(file_path):
            temp_df = pd.read_csv(file_path)[['Time_UTC', 'Load']]
            temp_df['Time_UTC'] = pd.to_datetime(temp_df['Time_UTC'])
            # Additional grouping by day
            temp_df['Day'] = temp_df['Time_UTC'].dt.day
            temp_df['Year'] = temp_df['Time_UTC'].dt.year
            temp_df['Month'] = temp_df['Time_UTC'].dt.month
            aggregated_df = temp_df.groupby(['Year', 'Month', 'Day'])['Load'].sum().reset_index()
            rb_df_list.append(aggregated_df)
    concatenated_df = pd.concat(rb_df_list, ignore_index=True)
    return concatenated_df
def aggregate_state_and_usa_daily(final_rb_df):
    # Read the state to rb mapping
    state_to_ba_mapping = pd.read_csv('/Users/ansonkong/Downloads/demand_prediciton_with_weather/resources/state_to_ba_mapping.csv')
    
    # For each state, sum the relevant 'rb' daily sums
    for index, row in state_to_ba_mapping.iterrows():
        rb_list = string_to_set(row['reeds_ba_list'])  # Convert string to list/set of rb codes
        state_column = row['state']
        # Ensure only existing columns are summed
        rb_columns = [rb for rb in rb_list if rb in final_rb_df.columns]
        # Summing for each state
        final_rb_df[state_column] = final_rb_df[rb_columns].sum(axis=1)
    
    # Summing all states to get USA total
    state_columns = state_to_ba_mapping['state'].tolist()
    final_rb_df['USA'] = final_rb_df[state_columns].sum(axis=1)
    
    return final_rb_df

def upper_95(x):
    return x.quantile(0.95)

def lower_5(x):
    return x.quantile(0.05)
# Define a function to reorganize each column into an array grouped by Year
def reorganize_into_array(series):
    # Pre-fill the array with NaNs to handle missing weekdays
    array = [np.nan] * 7  # One entry for each day of the week
    for idx, value in series.items():
        # idx is a tuple (Year, weekday), value is the column value
        # Subtract 1 from idx[1] if weekday starts from 1 in your dataset
        array[idx[1]] = value  # Or idx[1] - 1 if weekday is 1-based
    return array

final_dfs = []
for year in years:
    # Process data for the year (pseudo-code placeholders for actual processing)
    First=True
    for rb_code in rb_codes:
        temp_df = process_rb_files_daily(rb_code, [year])
        if First:
            processed_df=temp_df.copy()
            First=False
            processed_df[rb_code]=processed_df['Load']
            processed_df=processed_df.drop(['Load'], axis=1)
        else:
            # Assuming processed_df and temp_df are defined and have the appropriate columns
            processed_df = processed_df.merge(temp_df, on=['Year', 'Month', 'Day'], how='inner')
            processed_df[rb_code]=processed_df['Load']
            processed_df=processed_df.drop(['Load'], axis=1)
    processed_df = aggregate_state_and_usa_daily(processed_df)
    processed_df['Date'] = pd.to_datetime(processed_df[['Year', 'Month', 'Day']])

    # Create the 'weekday' column, note that dt.dayofweek returns Monday=0 through Sunday=6, so we add 1
    processed_df['weekday'] = processed_df['Date'].dt.dayofweek 
    processed_df=processed_df.drop(['Month','Day','Date'], axis=1)
    # Group by 'weekday' and 'Year', then apply the aggregations
    aggregations = {col: ['mean','max', upper_95, lower_5] for col in processed_df.columns if col not in ['Year', 'weekday']}
    grouped_df = processed_df.groupby(['Year', 'weekday']).agg(aggregations)
    # Now, flatten the MultiIndex in columns created by aggregation
    grouped_df.columns = ['{}_{}'.format(col[0], col[1]) for col in grouped_df.columns]

    # To further match your requirement, rename the aggregation methods in column names
    grouped_df.rename(columns=lambda x: x.replace('mean', 'mean').replace('upper_95', 'upper').replace('lower_5', 'lower'), inplace=True)

    # Reset index to turn 'Year' and 'weekday' back into columns if needed
    grouped_df= grouped_df.reset_index()
    # First, let's ensure 'weekday' is in the correct data type if not already
    grouped_df['weekday'] = grouped_df['weekday'].astype(int)
    final_dfs.append(grouped_df)
    

final_df = pd.concat(final_dfs, axis=0, ignore_index=True)
final_df.to_csv('/Users/ansonkong/Downloads/demand_prediciton_with_weather/resources/mock_rcp45hotter_weekly.csv')
            



In [134]:
import pandas as pd
import numpy as np
import os

# Define the directory where the CSV files are located and other initial setups
directory = '/Volumes/T7/prediction/rcp45hotter'
rb_codes = [f'p{i}' for i in range(1, 135)]
years = list(range(2020, 2100))

def string_to_set(string):
    elements = string.replace('{', '').replace('}', '').split(',')
    return set(e.strip().replace("'", "") for e in elements)

def upper_95(x):
    return x.quantile(0.95)

def lower_5(x):
    return x.quantile(0.05)

def process_rb_files_hourly(rb_code, year):
    file_name = f"{rb_code}_{year}_mlp_output.csv"
    file_path = os.path.join(directory, str(year), file_name)
    if os.path.isfile(file_path):
        temp_df = pd.read_csv(file_path, usecols=['Time_UTC', 'Load'])
        temp_df['Time_UTC'] = pd.to_datetime(temp_df['Time_UTC'])
        temp_df[rb_code]=temp_df['Load']
        # temp_df['Hour'] = temp_df['Time_UTC'].dt.hour
        # temp_df['Year'] = temp_df['Time_UTC'].dt.year
        # temp_df['Weekend_or_Weekday'] = np.where(temp_df['Time_UTC'].dt.dayofweek < 5, 'Weekday', 'Weekend')
        # temp_df['Day'] = temp_df['Time_UTC'].dt.day
        # temp_df['Month'] = temp_df['Time_UTC'].dt.month
        temp_df.drop(columns=['Load'], inplace=True)
        return temp_df
    else:
        return pd.DataFrame()

def aggregate_state_and_usa_daily(df, state_to_ba_mapping):
    for index, row in state_to_ba_mapping.iterrows():
        rb_list = string_to_set(row['reeds_ba_list'])
        state_column = row['state']
        df[state_column] = df[[col for col in df.columns if col in rb_list]].sum(axis=1)
    df['USA'] = df[[row['state'] for index, row in state_to_ba_mapping.iterrows()]].sum(axis=1)
    return df

def process_yearly_data(year):
    state_to_ba_mapping = pd.read_csv('/Users/ansonkong/Downloads/demand_prediciton_with_weather/resources/state_to_ba_mapping.csv')
    First = True

    for rb_code in rb_codes:
        temp_df = process_rb_files_hourly(rb_code, year)
        if First:
            processed_df=temp_df.copy()
            First=False
        else:
            processed_df = processed_df.merge(temp_df, on=['Time_UTC'], how='inner')
    yearly_data=processed_df 

    yearly_data = aggregate_state_and_usa_daily(yearly_data, state_to_ba_mapping)
    yearly_data['Hour'] = yearly_data['Time_UTC'].dt.hour
    yearly_data['Year'] = yearly_data['Time_UTC'].dt.year
    yearly_data['Weekend_or_Weekday'] = np.where(yearly_data['Time_UTC'].dt.dayofweek < 5, 'Weekday', 'Weekend')
    yearly_data.drop(columns=['Time_UTC'], inplace=True)
    columns_to_aggregate = [col for col in yearly_data.columns if col not in ['Hour', 'Year', 'Weekend_or_Weekday']]
    aggregations = {col: ['mean', upper_95, lower_5,'max'] for col in columns_to_aggregate}
        
    # Group by 'Hour', 'Year', 'Weekend_or_Weekday', then aggregate
    grouped_yearly_data = yearly_data.groupby(['Hour', 'Year', 'Weekend_or_Weekday']).agg(aggregations)
    # Flatten the MultiIndex columns if you have multiple columns being aggregated
    grouped_yearly_data.columns = ['_'.join(col).strip() for col in grouped_yearly_data.columns.values]

    # Reset index if you want 'Hour', 'Year', 'Weekend_or_Weekday' back as regular columns
    grouped_yearly_data.reset_index(inplace=True)
    grouped_yearly_data.rename(columns=lambda x: x.replace('mean', 'mean').replace('upper_95', 'upper').replace('lower_5', 'lower'), inplace=True)

    return grouped_yearly_data

final_dfs = []
for year in years:
    yearly_data = process_yearly_data(year)
    if not yearly_data.empty:
        final_dfs.append(yearly_data)

final_df = pd.concat(final_dfs, ignore_index=True)
final_df.to_csv('/Users/ansonkong/Downloads/demand_prediciton_with_weather/resources/mock_rcp45hotter_yearly_aggregated.csv')
final_df

/var/folders/8v/68yqll_54lzgb57b5q8_fbrc0000gn/T/ipykernel_43764/832301105.py:72: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  grouped_yearly_data.reset_index(inplace=True)
/var/folders/8v/68yqll_54lzgb57b5q8_fbrc0000gn/T/ipykernel_43764/832301105.py:72: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  grouped_yearly_data.reset_index(inplace=True)
/var/folders/8v/68yqll_54lzgb57b5q8_fbrc0000gn/T/ipykernel_43764/832301105.py:72: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert

,Hour,Year,Weekend_or_Weekday,p1_mean,p1_upper,p1_lower,p1_max,p2_mean,p2_upper,p2_lower,...,Wisconsin_lower,Wisconsin_max,Wyoming_mean,Wyoming_upper,Wyoming_lower,Wyoming_max,USA_mean,USA_upper,USA_lower,USA_max
0,0,2020,Weekday,4242.825019,5330.1600,3631.7700,6814.02,2520.231303,3086.0800,2216.6300,...,6208.3500,8737.21,1876.569004,2166.8100,1642.9000,2250.83,367352.999349,437336.5100,316290.2000,463603.02
1,0,2020,Weekend,4284.082596,5692.4420,3629.0015,6192.61,2513.256827,3173.5310,2227.8430,...,5812.8905,7913.08,1819.073173,2074.0225,1578.1800,2185.67,361006.065385,432374.4915,311061.3985,439356.20
2,1,2020,Weekday,4157.650229,5251.6750,3541.9760,6453.90,2483.642061,3026.0240,2148.8885,...,6182.8570,8531.39,1869.258779,2157.9700,1630.5570,2273.24,361061.103053,422086.8355,312662.6960,441500.76
3,1,2020,Weekend,4137.083750,5428.9195,3487.0030,6069.46,2469.460673,3160.5475,2170.1660,...,5722.8730,7726.19,1814.317981,2061.8910,1572.6440,2206.31,350635.470481,415939.1540,303049.1325,431039.55
4,2,2020,Weekday,4106.464084,5209.1150,3455.9820,6569.08,2481.141718,3125.1760,2160.2645,...,6184.2545,8437.42,1858.261832,2119.6555,1614.7055,2296.20,358610.104885,417297.6845,311768.6080,445401.73
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3835,21,2099,Weekend,5281.749423,6188.5085,4725.4365,6396.41,2978.595673,3325.4635,2672.9450,...,6426.4595,10669.98,1993.642308,2287.4615,1714.3550,2373.06,438023.447212,544269.3390,376798.7880,577088.09
3836,22,2099,Weekday,5165.849387,6095.4400,4503.8500,6571.63,2940.345747,3348.0000,2576.8600,...,6789.7100,10662.58,2016.116590,2357.8300,1733.2300,2471.95,432224.909387,536734.2800,370453.4500,564738.59
3837,22,2099,Weekend,4975.109808,5822.9850,4458.0565,6055.02,2813.782019,3150.3545,2480.5945,...,6231.9030,10259.83,1945.690865,2243.5215,1669.5330,2329.85,412493.102500,509333.8205,357118.0775,544440.41
3838,23,2099,Weekday,4761.743372,5606.7600,4177.0100,6062.68,2739.847011,3093.6400,2417.7200,...,6447.7800,10034.45,1944.355824,2278.4200,1683.2800,2385.01,402546.938084,496214.6500,345187.9200,524610.60


In [24]:
import os

directory = '/Users/ansonkong/Downloads/Data for nyu work/output/demand_weather_combine_2007'
expected_files = [f'p{i}_historical_data.csv' for i in range(1, 135)]
missing_files = [file for file in expected_files if not os.path.isfile(os.path.join(directory, file))]

if missing_files:
    print("Missing files:", missing_files)
else:
    print("All files are present.")

import pandas as pd
import numpy as np
import scipy.stats as st

# Function to calculate the mean, SD, and 95% CI
def calculate_stats(df, group_col, calc_cols):
    grouped = df.groupby(group_col)[calc_cols].agg(['mean', 'std']).reset_index()
    
    # Calculate 95% CI
    for col in calc_cols:
        ci_low, ci_high = st.t.interval(alpha=0.95, df=len(df[col])-1, loc=np.mean(df[col]), scale=st.sem(df[col]))
        grouped[(col, '95%_ci_low')] = ci_low
        grouped[(col, '95%_ci_high')] = ci_high
    
    return grouped

# Columns to calculate statistics for
calc_cols = ['T2',  'Adjusted_Demand_MWh']

# Initialize DataFrame to hold combined results
combined_results = pd.DataFrame()

# Process each file
for rb_file in expected_files:
    file_path = os.path.join(directory, rb_file)
    if os.path.isfile(file_path):
        df = pd.read_csv(file_path)
        
        # Assuming 'Month' is already correctly formatted
        stats_df = calculate_stats(df, 'Month', calc_cols)
        
        # Add identifier for the rb_file
        stats_df['rb'] = rb_file.split('_')[0]
        
        combined_results = pd.concat([combined_results, stats_df], ignore_index=True)

# Ensure column names are properly formatted after aggregation and CI calculation
combined_results.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in combined_results.columns]
output_path = '/Users/ansonkong/Downloads/Data for nyu work/output/combined_stats.csv'
combined_results.to_csv(output_path, index=False)
# Step 1: Rename 'Month_' and 'rb_' to 'Month' and 'rb'
combined_results.rename(columns=lambda x: x.replace('Month_', 'Month').replace('rb_', 'rb'), inplace=True)

# Step 2: Filter out columns to keep only CI columns, 'Month', and 'rb'
# Identify CI columns by checking if '95%_ci' is part of the column name
ci_columns = [col for col in combined_results.columns if '95%_ci' in col]
required_columns = ['Month', 'rb'] + ci_columns

# Create a new DataFrame with only the required columns
filtered_results = combined_results[required_columns]


All files are present.


/var/folders/8v/68yqll_54lzgb57b5q8_fbrc0000gn/T/ipykernel_43764/90775692.py:22: DeprecationWarning: Use of keyword argument `alpha` for method `interval` is deprecated. Use first positional argument or keyword argument `confidence` instead.
  ci_low, ci_high = st.t.interval(alpha=0.95, df=len(df[col])-1, loc=np.mean(df[col]), scale=st.sem(df[col]))
/var/folders/8v/68yqll_54lzgb57b5q8_fbrc0000gn/T/ipykernel_43764/90775692.py:22: DeprecationWarning: Use of keyword argument `alpha` for method `interval` is deprecated. Use first positional argument or keyword argument `confidence` instead.
  ci_low, ci_high = st.t.interval(alpha=0.95, df=len(df[col])-1, loc=np.mean(df[col]), scale=st.sem(df[col]))
/var/folders/8v/68yqll_54lzgb57b5q8_fbrc0000gn/T/ipykernel_43764/90775692.py:22: DeprecationWarning: Use of keyword argument `alpha` for method `interval` is deprecated. Use first positional argument or keyword argument `confidence` instead.
  ci_low, ci_high = st.t.interval(alpha=0.95, df=len(d

In [26]:
combined_results
# Step 1: Rename 'Month_' and 'rb_' to 'Month' and 'rb'
combined_results.rename(columns=lambda x: x.replace('Month_', 'Month').replace('rb_', 'rb'), inplace=True)

# Step 2: Filter out columns to keep only CI columns, 'Month', and 'rb'
# Identify CI columns by checking if '95%_ci' is part of the column name
ci_columns = [col for col in combined_results.columns if '95%_ci' in col]
required_columns = ['Month', 'rb'] + ci_columns

# Create a new DataFrame with only the required columns
filtered_results = combined_results[required_columns]

print(filtered_results)


      Month    rb  T2_95%_ci_low  T2_95%_ci_high  Q2_95%_ci_low  \
0         1    p1     280.321848      280.443071       0.005339   
1         2    p1     280.321848      280.443071       0.005339   
2         3    p1     280.321848      280.443071       0.005339   
3         4    p1     280.321848      280.443071       0.005339   
4         5    p1     280.321848      280.443071       0.005339   
...     ...   ...            ...             ...            ...   
1603      8  p134     278.558307      278.753295       0.005618   
1604      9  p134     278.558307      278.753295       0.005618   
1605     10  p134     278.558307      278.753295       0.005618   
1606     11  p134     278.558307      278.753295       0.005618   
1607     12  p134     278.558307      278.753295       0.005618   

      Q2_95%_ci_high  SWDOWN_95%_ci_low  SWDOWN_95%_ci_high  GLW_95%_ci_low  \
0           0.005376         161.283151          165.529478      294.460197   
1           0.005376         161.2831

In [31]:
import pandas as pd
import os
def get_ci_for_rb_month(filtered_results, rb,  month):
    # Filter for the specific rb and month
    ci_row = filtered_results[(filtered_results['rb'] == rb) & (filtered_results['Month'] == month)]
    
    # Extract the CI lower and upper bounds for T2
    # Assuming CI column names follow a specific pattern like 'T2_95%_ci_low' and 'T2_95%_ci_high'
    ci_lower = ci_row['T2_95%_ci_low'].values[0] if not ci_row.empty else None
    ci_upper = ci_row['T2_95%_ci_high'].values[0] if not ci_row.empty else None
    
    return ci_lower, ci_upper
import os

directory = '/Users/ansonkong/Downloads/Data for nyu work/output/demand_weather_combine_2007'


import pandas as pd
import numpy as np
import scipy.stats as st

# Function to calculate the mean, SD, and 95% CI
def calculate_stats(df, group_col, calc_cols):
    grouped = df.groupby(group_col)[calc_cols].agg(['mean', 'std']).reset_index()
    
    # Calculate 95% CI
    for col in calc_cols:
        ci_low, ci_high = st.t.interval(alpha=0.95, df=len(df[col])-1, loc=np.mean(df[col]), scale=st.sem(df[col]))
        grouped[(col, '95%_ci_low')] = ci_low
        grouped[(col, '95%_ci_high')] = ci_high
    
    return grouped

# Columns to calculate statistics for
calc_cols = ['T2',  'Adjusted_Demand_MWh']

# Initialize DataFrame to hold combined results
combined_results = pd.DataFrame()

# Process each file
for rb_file in expected_files:
    file_path = os.path.join(directory, rb_file)
    if os.path.isfile(file_path):
        df = pd.read_csv(file_path)
        
        # Assuming 'Month' is already correctly formatted
        stats_df = calculate_stats(df, 'Month', calc_cols)
        
        # Add identifier for the rb_file
        stats_df['rb'] = rb_file.split('_')[0]
        
        combined_results = pd.concat([combined_results, stats_df], ignore_index=True)

# Ensure column names are properly formatted after aggregation and CI calculation
combined_results.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in combined_results.columns]
output_path = '/Users/ansonkong/Downloads/Data for nyu work/output/combined_stats.csv'
combined_results.to_csv(output_path, index=False)
# Step 1: Rename 'Month_' and 'rb_' to 'Month' and 'rb'
combined_results.rename(columns=lambda x: x.replace('Month_', 'Month').replace('rb_', 'rb'), inplace=True)

# Step 2: Filter out columns to keep only CI columns, 'Month', and 'rb'
# Identify CI columns by checking if '95%_ci' is part of the column name
ci_columns = [col for col in combined_results.columns if '95%_ci' in col]
required_columns = ['Month', 'rb'] + ci_columns

# Create a new DataFrame with only the required columns
filtered_results = combined_results[required_columns]


# Assuming combined_results is prepared as described
# Filter for T2-related confidence interval columns
ci_columns = [col for col in combined_results.columns if 'T2_95%_ci' in col]
required_columns = ['Month', 'rb'] + ci_columns
filtered_results = combined_results[required_columns]

# Initialize a DataFrame to store instances outside CI
outliers = pd.DataFrame()

directory = '/Users/ansonkong/Downloads/Data for nyu work/output/future_rcp85hotter_weather'
rb_codes = [f'p{i}' for i in range(1, 135)]
years = range(2020, 2100)
output_directory = '/Users/ansonkong/Downloads/demand_prediciton_with_weather/resources/outlier'  # Update this to your desired output directory
# Make sure the output directory exists
os.makedirs(output_directory, exist_ok=True)


for rb in rb_codes:
    # Initialize a DataFrame to store instances outside CI
    outliers = pd.DataFrame()
    for year in years:
        for month in range(1, 13):  # Assuming you want to check all months
            file_path = os.path.join(directory, f'{rb}_WRF_Hourly_Mean_Meteorology_{year}.csv')
            if os.path.isfile(file_path):
                df = pd.read_csv(file_path)
                df['Month'] = pd.to_datetime(df['Time_UTC']).dt.month  # Ensure 'Month' column exists
                # Filter df for the current month in iteration
                df_month = df[df['Month'] == month]
                if not df_month.empty:
                    ci_lower, ci_upper = get_ci_for_rb_month(filtered_results, rb, month)
                    if ci_lower is not None and ci_upper is not None:
                        below = df_month[df_month['T2'] < ci_lower]
                        above = df_month[df_month['T2'] > ci_upper]
                        # Set 'status' directly
                        below['status'] = 'below'
                        above['status'] = 'above'
                        # Add 'rb' and concatenate
                        for d in [below, above]:
                            d['rb'] = rb
                            outliers = pd.concat([outliers, d[['Time_UTC', 'status', 'rb']]], ignore_index=True)
    # Define the output file path
    output_file_path = os.path.join(output_directory, f'{rb}_outliers.csv')
    
    # Save the filtered DataFrame to a CSV file
    outliers.to_csv(output_file_path, index=False)

outliers

/var/folders/8v/68yqll_54lzgb57b5q8_fbrc0000gn/T/ipykernel_43764/3199683495.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  below['status'] = 'below'
/var/folders/8v/68yqll_54lzgb57b5q8_fbrc0000gn/T/ipykernel_43764/3199683495.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  above['status'] = 'above'
/var/folders/8v/68yqll_54lzgb57b5q8_fbrc0000gn/T/ipykernel_43764/3199683495.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [68]:
import pandas as pd
import numpy as np
import scipy.stats as st
import os

# Define the directory where your CSV files are located
directory = '/Users/ansonkong/Downloads/Data for nyu work/output/demand_weather_combine_2007'
ci_output_directory = os.path.join('/Users/ansonkong/Downloads/demand_prediciton_with_weather/resources', 'CI_results')
os.makedirs(ci_output_directory, exist_ok=True)  # Ensure the output directory exists

def calculate_quantile_ci(data, confidence=0.95):
    """
    Calculate the confidence interval using quantiles for a given dataset.
    """
    # Calculate the lower and upper percentile bounds for the confidence interval
    lower_percentile = 100 * (1 - confidence) / 2
    upper_percentile = 100 - lower_percentile
    
    # Use np.percentile to find the lower and upper quantiles
    ci_lower = np.percentile(data, lower_percentile)
    ci_upper = np.percentile(data, upper_percentile)
    
    return ci_lower, ci_upper

# List all files in the directory
files = [file for file in os.listdir(directory) if file.endswith('.csv')]

for file in files:
    filepath = os.path.join(directory, file)
    df = pd.read_csv(filepath)
    
    # Extract 'rb' from filename
    rb_code = file.split('_')[0]
    
    # Convert Time_UTC to datetime and extract Year, Month, and Day
    df['Time_UTC'] = pd.to_datetime(df['Time_UTC'])
    df['Month'] = df['Time_UTC'].dt.month
    
    # Initialize lists to store CI results
    ci_results_max = []
    ci_results_min = []
    
    for month in range(1, 13):
        month_data = df[df['Month'] == month]
        daily_max = month_data.groupby(month_data['Time_UTC'].dt.date)['T2'].max()
        daily_min = month_data.groupby(month_data['Time_UTC'].dt.date)['T2'].min()
        
        ci_max_lower, ci_max_upper = calculate_quantile_ci(daily_max)
        ci_min_lower, ci_min_upper = calculate_quantile_ci(daily_min)
        
        ci_results_max.append([rb_code, ci_max_lower, ci_max_upper])
        ci_results_min.append([rb_code, ci_min_lower, ci_min_upper])
    
    # Convert CI results to DataFrames
    ci_df_max = pd.DataFrame(ci_results_max, columns=['rb',  'CI_max_lower', 'CI_max_upper'])
    ci_df_min = pd.DataFrame(ci_results_min, columns=['rb',  'CI_min_lower', 'CI_min_upper'])
    # For ci_df_max, keep only the maximum upper CI for each rb
    ci_df_max_reduced = ci_df_max.groupby('rb')['CI_max_upper'].max().reset_index()

    # For ci_df_min, keep only the minimum lower CI for each rb
    ci_df_min_reduced = ci_df_min.groupby('rb')['CI_min_lower'].min().reset_index()

    # Reindex the DataFrames
    # ci_df_max_reduced.reset_index(drop=True, inplace=True)
    # ci_df_min_reduced.reset_index(drop=True, inplace=True)
    
    # Save CI DataFrames to CSV files
    ci_df_max_reduced.to_csv(os.path.join(ci_output_directory, f'{rb_code}_CI_max.csv'), index=False)
    ci_df_min_reduced.to_csv(os.path.join(ci_output_directory, f'{rb_code}_CI_min.csv'), index=False)


In [96]:
import pandas as pd
import numpy as np
import os

# Directories initialization
directory = '/Users/ansonkong/Downloads/Data for nyu work/output/future_rcp85hotter_weather'
ci_directory = '/Users/ansonkong/Downloads/demand_prediciton_with_weather/resources/CI_results'
output_directory = '/Users/ansonkong/Downloads/demand_prediciton_with_weather/resources/outlier'
os.makedirs(output_directory, exist_ok=True)

rb_codes = [f'p{i}' for i in range(1, 135)]
years = range(2020, 2100)

extreme_outliers_max_list = []
extreme_outliers_min_list = []

for rb_code in rb_codes:
    extreme_outliers_max_list = []
    extreme_outliers_min_list = []
    ci_max_path = os.path.join(ci_directory, f'{rb_code}_CI_max.csv')
    ci_min_path = os.path.join(ci_directory, f'{rb_code}_CI_min.csv')
    
    if os.path.exists(ci_max_path) and os.path.exists(ci_min_path):
        ci_max_df = pd.read_csv(ci_max_path)
        ci_min_df = pd.read_csv(ci_min_path)
        
        for year in years:
            weather_file_path = os.path.join(directory, f'{rb_code}_WRF_Hourly_Mean_Meteorology_{year}.csv')
            if os.path.exists(weather_file_path):
                df = pd.read_csv(weather_file_path)
                df['Date'] = pd.to_datetime(df['Time_UTC']).dt.date
                df['Month'] = pd.to_datetime(df['Time_UTC']).dt.month
                    
                # Only proceed if the month matches
                ci_max_month = ci_max_df[ (ci_max_df['rb'] == rb_code)]
                ci_min_month = ci_min_df[ (ci_min_df['rb'] == rb_code)]
                    
                # Find the dates with the highest and lowest T2 for each day
                grouped = df.groupby('Date')['T2'].agg(['max', 'min'])
                dates_above_max_ci = grouped[grouped['max'] > ci_max_month['CI_max_upper'].iloc[0]].index.tolist()
                # print(ci_max_df)
                # print(grouped)
                # print(grouped[grouped['max'] > ci_max_month['CI_max_upper'].iloc[0]].index.tolist())
                for date in dates_above_max_ci:
                    extreme_outliers_max_list.append({'rb': rb_code, 'Year': year, 'Date': date})

                # Find all dates where min T2 is below the CI_min_lower
                dates_below_min_ci = grouped[grouped['min'] < ci_min_month['CI_min_lower'].iloc[0]].index.tolist()
                for date in dates_below_min_ci:
                    extreme_outliers_min_list.append({'rb': rb_code, 'Year': year, 'Date': date})

    # Convert lists to DataFrames
    extreme_outliers_max = pd.DataFrame(extreme_outliers_max_list)
    extreme_outliers_min = pd.DataFrame(extreme_outliers_min_list)

    # Save the results
    extreme_outliers_max.to_csv(os.path.join(output_directory, f'{rb_code}_extreme_outliers_max.csv'), index=False)
    extreme_outliers_min.to_csv(os.path.join(output_directory, f'{rb_code}_extreme_outliers_min.csv'), index=False)


In [75]:
import pandas as pd
import numpy as np
import scipy.stats as st
import os

# Define the directory where your CSV files are located
directory = '/Users/ansonkong/Downloads/Data for nyu work/output/demand_weather_combine_2007'
ci_output_directory = os.path.join('/Users/ansonkong/Downloads/demand_prediciton_with_weather/resources', 'CI_results')
os.makedirs(ci_output_directory, exist_ok=True)  # Ensure the output directory exists

def calculate_quantile_ci(data, confidence=0.95):
    """
    Calculate the confidence interval using quantiles for a given dataset.
    """
    # Calculate the lower and upper percentile bounds for the confidence interval
    lower_percentile = 100 * (1 - confidence) / 2
    upper_percentile = 100 - lower_percentile
    
    # Use np.percentile to find the lower and upper quantiles
    ci_lower = np.percentile(data, lower_percentile)
    ci_upper = np.percentile(data, upper_percentile)
    
    return ci_lower, ci_upper

# List all files in the directory
files = [file for file in os.listdir(directory) if file.endswith('.csv')]
states=['Alabama', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware', 'Florida', 'Georgia',
           'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland',
          'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey',
          'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina',
          'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming','usa']
rb_codes = [f'p{i}' for i in range(1, 135)]
for state in states:
    state_df=pd.DataFrame()
    for year in range(2007,2013):
        file=f'{state}_averaged_weather_{year}.csv'

        filepath = os.path.join('/Users/ansonkong/Downloads/Data for nyu work/averaged_historical_weather', file)
        df = pd.read_csv(filepath)
        # Concatenate this DataFrame with the accumulating state_df
        state_df = pd.concat([state_df, df], ignore_index=True)
    
    
    # Convert Time_UTC to datetime and extract Year, Month, and Day
    state_df['Time_UTC'] = pd.to_datetime(state_df['Time_UTC'])
    state_df['Month'] = state_df['Time_UTC'].dt.month
    
    # Initialize lists to store CI results
    ci_results_max = []
    ci_results_min = []
    
    for month in range(1, 13):
        month_data = state_df[state_df['Month'] == month]
        daily_max = month_data.groupby(month_data['Time_UTC'].dt.date)['T2'].max()
        daily_min = month_data.groupby(month_data['Time_UTC'].dt.date)['T2'].min()
        
        ci_max_lower, ci_max_upper = calculate_quantile_ci(daily_max)
        ci_min_lower, ci_min_upper = calculate_quantile_ci(daily_min)
        
        ci_results_max.append([ state, ci_max_lower, ci_max_upper])
        ci_results_min.append([ state, ci_min_lower, ci_min_upper])
    
    # Convert CI results to DataFrames
    ci_df_max = pd.DataFrame(ci_results_max, columns=['State',  'CI_max_lower', 'CI_max_upper'])
    ci_df_min = pd.DataFrame(ci_results_min, columns=['State',  'CI_min_lower', 'CI_min_upper'])
    # For ci_df_max, keep only the maximum upper CI for each rb
    ci_df_max_reduced = ci_df_max.groupby('State')['CI_max_upper'].max().reset_index()

    # For ci_df_min, keep only the minimum lower CI for each rb
    ci_df_min_reduced = ci_df_min.groupby('State')['CI_min_lower'].min().reset_index()

    # Reindex the DataFrames
    # ci_df_max_reduced.reset_index(drop=True, inplace=True)
    # ci_df_min_reduced.reset_index(drop=True, inplace=True)
    
    # Save CI DataFrames to CSV files
    ci_df_max_reduced.to_csv(os.path.join(ci_output_directory, f'{state}_CI_max.csv'), index=False)
    ci_df_min_reduced.to_csv(os.path.join(ci_output_directory, f'{state}_CI_min.csv'), index=False)

In [95]:
import pandas as pd
import numpy as np
import os

# Directories initialization
directory = '/Users/ansonkong/Downloads/Data for nyu work/averaged_rcp85hotter_weather'
ci_directory = '/Users/ansonkong/Downloads/demand_prediciton_with_weather/resources/CI_results'
output_directory = '/Users/ansonkong/Downloads/demand_prediciton_with_weather/resources/outlier'
os.makedirs(output_directory, exist_ok=True)

states=['Alabama', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware', 'Florida', 'Georgia',
           'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland',
          'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey',
          'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina',
          'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming','usa']
years = range(2020, 2100)

extreme_outliers_max_list = []
extreme_outliers_min_list = []

for state in states:
    extreme_outliers_max_list = []
    extreme_outliers_min_list = []
    ci_max_path = os.path.join(ci_directory, f'{state}_CI_max.csv')
    ci_min_path = os.path.join(ci_directory, f'{state}_CI_min.csv')
    
    if os.path.exists(ci_max_path) and os.path.exists(ci_min_path):
        ci_max_df = pd.read_csv(ci_max_path)
        ci_min_df = pd.read_csv(ci_min_path)
        
        for year in years:
            weather_file_path = os.path.join(directory, f'{state}_averaged_weather_{year}.csv')
            if os.path.exists(weather_file_path):
                df = pd.read_csv(weather_file_path)
                df['Date'] = pd.to_datetime(df['Time_UTC']).dt.date
                df['Month'] = pd.to_datetime(df['Time_UTC']).dt.month
                    
                # Only proceed if the month matches
                ci_max_month = ci_max_df[ (ci_max_df['State'] == state)]
                ci_min_month = ci_min_df[ (ci_min_df['State'] == state)]
                    
                # Find the dates with the highest and lowest T2 for each day
                grouped = df.groupby('Date')['T2'].agg(['max', 'min'])
                dates_above_max_ci = grouped[grouped['max'] > ci_max_month['CI_max_upper'].iloc[0]].index.tolist()
                # print(ci_max_df)
                # print(grouped)
                # print(grouped[grouped['max'] > ci_max_month['CI_max_upper'].iloc[0]].index.tolist())
                for date in dates_above_max_ci:
                    extreme_outliers_max_list.append({'rb': rb_code, 'Year': year, 'Date': date})

                # Find all dates where min T2 is below the CI_min_lower
                dates_below_min_ci = grouped[grouped['min'] < ci_min_month['CI_min_lower'].iloc[0]].index.tolist()
                for date in dates_below_min_ci:
                    extreme_outliers_min_list.append({'rb': rb_code, 'Year': year, 'Date': date})

    # Convert lists to DataFrames
    extreme_outliers_max = pd.DataFrame(extreme_outliers_max_list)
    extreme_outliers_min = pd.DataFrame(extreme_outliers_min_list)

    # Save the results
    extreme_outliers_max.to_csv(os.path.join(output_directory, f'{state}_extreme_outliers_max.csv'), index=False)
    extreme_outliers_min.to_csv(os.path.join(output_directory, f'{state}_extreme_outliers_min.csv'), index=False)

In [97]:
import os
import pandas as pd

# Specify the folder
folder = '/Users/ansonkong/Downloads/demand_prediciton_with_weather/resources/outlier'

# Initialize an empty DataFrame to store all results
all_results_df = pd.DataFrame()

# List all "max" files in the folder
max_files = [f for f in os.listdir(folder) if f.endswith('_extreme_outliers_max.csv')]

for file in max_files:
    # Construct the full file path
    filepath = os.path.join(folder, file)
    
    # Extract the region from the file name
    region = file.split('_extreme_outliers_max.csv')[0]
    
    try:
        # Attempt to read the file into a DataFrame
        df = pd.read_csv(filepath)
    except pd.errors.EmptyDataError:
        print(f"Skipping empty file: {file}")
        continue 
    # Group by 'Year' and count the number of observations for each year
    year_counts = df.groupby('Year').size().reset_index(name='number_of_days')
    
    # Add the region to the DataFrame
    year_counts['region'] = region
    
    # Append the result to the all_results_df DataFrame
    all_results_df = pd.concat([all_results_df, year_counts], ignore_index=True)

# Reorder the DataFrame columns if needed
all_results_df = all_results_df[['region', 'Year', 'number_of_days']]

# You now have a DataFrame containing the region, Year, and number_of_days for all "max" files
# Optionally, save this DataFrame to a new CSV file
all_results_df.to_csv('/Users/ansonkong/Downloads/demand_prediciton_with_weather/resources/all_max_outliers_summary_rcp85hotter.csv', index=False)


In [94]:
import os
import pandas as pd

# Specify the folder
folder = '/Users/ansonkong/Downloads/demand_prediciton_with_weather/resources/outlier'

# Initialize an empty DataFrame to store all results for minimum outliers
all_min_results_df = pd.DataFrame()

# List all "min" files in the folder
min_files = [f for f in os.listdir(folder) if f.endswith('_extreme_outliers_min.csv')]

for file in min_files:
    # Construct the full file path
    filepath = os.path.join(folder, file)
    
    # Extract the region from the file name
    region = file.split('_extreme_outliers_min.csv')[0]
    
    try:
        # Attempt to read the file into a DataFrame
        df = pd.read_csv(filepath)
    except pd.errors.EmptyDataError:
        print(f"Skipping empty file: {file}")
        continue 
    
    # Group by 'Year' and count the number of observations for each year
    year_counts = df.groupby('Year').size().reset_index(name='number_of_days')
    
    # Add the region to the DataFrame
    year_counts['region'] = region
    
    # Append the result to the all_min_results_df DataFrame
    all_min_results_df = pd.concat([all_min_results_df, year_counts], ignore_index=True)

# Reorder the DataFrame columns if needed
all_min_results_df = all_min_results_df[['region', 'Year', 'number_of_days']]

# You now have a DataFrame containing the region, Year, and number_of_days for all "min" files
# Optionally, save this DataFrame to a new CSV file
all_min_results_df.to_csv('/Users/ansonkong/Downloads/demand_prediciton_with_weather/resources/all_min_outliers_summary_rcp85hotter.csv', index=False)


In [135]:
import pandas as pd
import numpy as np
import os
for case in ['rcp45hotter']:#, 'rcp45hotter', 'rcp45cooler']
    # Directories initialization
    directory = f'/Users/ansonkong/Downloads/Data for nyu work/output/future_{case}_weather'
    ci_directory = '/Users/ansonkong/Downloads/demand_prediciton_with_weather/resources/CI_results'
    output_directory = '/Users/ansonkong/Downloads/demand_prediciton_with_weather/resources/outlier'
    os.makedirs(output_directory, exist_ok=True)

    rb_codes = [f'p{i}' for i in range(1, 135)]
    years = range(2020, 2100)

    extreme_outliers_max_list = []
    extreme_outliers_min_list = []

    for rb_code in rb_codes:
        extreme_outliers_max_list = []
        extreme_outliers_min_list = []
        ci_max_path = os.path.join(ci_directory, f'{rb_code}_CI_max.csv')
        ci_min_path = os.path.join(ci_directory, f'{rb_code}_CI_min.csv')
        
        if os.path.exists(ci_max_path) and os.path.exists(ci_min_path):
            ci_max_df = pd.read_csv(ci_max_path)
            ci_min_df = pd.read_csv(ci_min_path)
            
            for year in years:
                weather_file_path = os.path.join(directory, f'{rb_code}_WRF_Hourly_Mean_Meteorology_{year}.csv')
                if os.path.exists(weather_file_path):
                    df = pd.read_csv(weather_file_path)
                    df['Date'] = pd.to_datetime(df['Time_UTC']).dt.date
                    df['Month'] = pd.to_datetime(df['Time_UTC']).dt.month
                        
                    # Only proceed if the month matches
                    ci_max_month = ci_max_df[ (ci_max_df['rb'] == rb_code)]
                    ci_min_month = ci_min_df[ (ci_min_df['rb'] == rb_code)]
                        
                    # Find the dates with the highest and lowest T2 for each day
                    grouped = df.groupby('Date')['T2'].agg(['max', 'min'])
                    dates_above_max_ci = grouped[grouped['max'] > ci_max_month['CI_max_upper'].iloc[0]].index.tolist()
                    # print(ci_max_df)
                    # print(grouped)
                    # print(grouped[grouped['max'] > ci_max_month['CI_max_upper'].iloc[0]].index.tolist())
                    for date in dates_above_max_ci:
                        extreme_outliers_max_list.append({'rb': rb_code, 'Year': year, 'Date': date})

                    # Find all dates where min T2 is below the CI_min_lower
                    dates_below_min_ci = grouped[grouped['min'] < ci_min_month['CI_min_lower'].iloc[0]].index.tolist()
                    for date in dates_below_min_ci:
                        extreme_outliers_min_list.append({'rb': rb_code, 'Year': year, 'Date': date})

        # Convert lists to DataFrames
        extreme_outliers_max = pd.DataFrame(extreme_outliers_max_list)
        extreme_outliers_min = pd.DataFrame(extreme_outliers_min_list)

        # Save the results
        extreme_outliers_max.to_csv(os.path.join(output_directory, f'{rb_code}_extreme_outliers_max_{case}.csv'), index=False)
        extreme_outliers_min.to_csv(os.path.join(output_directory, f'{rb_code}_extreme_outliers_min_{case}.csv'), index=False)


    # Directories initialization
    directory = f'/Users/ansonkong/Downloads/Data for nyu work/averaged_{case}_weather'
    ci_directory = '/Users/ansonkong/Downloads/demand_prediciton_with_weather/resources/CI_results'
    output_directory = '/Users/ansonkong/Downloads/demand_prediciton_with_weather/resources/outlier'
    os.makedirs(output_directory, exist_ok=True)

    states=['Alabama', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware', 'Florida', 'Georgia',
            'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland',
            'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey',
            'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina',
            'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming','usa']
    years = range(2020, 2100)

    extreme_outliers_max_list = []
    extreme_outliers_min_list = []

    for state in states:
        extreme_outliers_max_list = []
        extreme_outliers_min_list = []
        ci_max_path = os.path.join(ci_directory, f'{state}_CI_max.csv')
        ci_min_path = os.path.join(ci_directory, f'{state}_CI_min.csv')
        
        if os.path.exists(ci_max_path) and os.path.exists(ci_min_path):
            ci_max_df = pd.read_csv(ci_max_path)
            ci_min_df = pd.read_csv(ci_min_path)
            
            for year in years:
                weather_file_path = os.path.join(directory, f'{state}_averaged_weather_{year}.csv')
                if os.path.exists(weather_file_path):
                    df = pd.read_csv(weather_file_path)
                    df['Date'] = pd.to_datetime(df['Time_UTC']).dt.date
                    df['Month'] = pd.to_datetime(df['Time_UTC']).dt.month
                        
                    # Only proceed if the month matches
                    ci_max_month = ci_max_df[ (ci_max_df['State'] == state)]
                    ci_min_month = ci_min_df[ (ci_min_df['State'] == state)]
                        
                    # Find the dates with the highest and lowest T2 for each day
                    grouped = df.groupby('Date')['T2'].agg(['max', 'min'])
                    dates_above_max_ci = grouped[grouped['max'] > ci_max_month['CI_max_upper'].iloc[0]].index.tolist()
                    # print(ci_max_df)
                    # print(grouped)
                    # print(grouped[grouped['max'] > ci_max_month['CI_max_upper'].iloc[0]].index.tolist())
                    for date in dates_above_max_ci:
                        extreme_outliers_max_list.append({'rb': rb_code, 'Year': year, 'Date': date})

                    # Find all dates where min T2 is below the CI_min_lower
                    dates_below_min_ci = grouped[grouped['min'] < ci_min_month['CI_min_lower'].iloc[0]].index.tolist()
                    for date in dates_below_min_ci:
                        extreme_outliers_min_list.append({'rb': rb_code, 'Year': year, 'Date': date})

        # Convert lists to DataFrames
        extreme_outliers_max = pd.DataFrame(extreme_outliers_max_list)
        extreme_outliers_min = pd.DataFrame(extreme_outliers_min_list)

        # Save the results
        extreme_outliers_max.to_csv(os.path.join(output_directory, f'{state}_extreme_outliers_max_{case}.csv'), index=False)
        extreme_outliers_min.to_csv(os.path.join(output_directory, f'{state}_extreme_outliers_min_{case}.csv'), index=False)


    # Specify the folder
    folder = '/Users/ansonkong/Downloads/demand_prediciton_with_weather/resources/outlier'

    # Initialize an empty DataFrame to store all results
    all_results_df = pd.DataFrame()

    # List all "max" files in the folder
    max_files = [f for f in os.listdir(folder) if f.endswith(f'_extreme_outliers_max_{case}.csv')]

    for file in max_files:
        # Construct the full file path
        filepath = os.path.join(folder, file)
        
        # Extract the region from the file name
        region = file.split(f'_extreme_outliers_max_{case}.csv')[0]
        
        try:
            # Attempt to read the file into a DataFrame
            df = pd.read_csv(filepath)
        except pd.errors.EmptyDataError:
            print(f"Skipping empty file: {file}")
            continue 
        # Group by 'Year' and count the number of observations for each year
        year_counts = df.groupby('Year').size().reset_index(name='number_of_days')
        
        # Add the region to the DataFrame
        year_counts['region'] = region
        
        # Append the result to the all_results_df DataFrame
        all_results_df = pd.concat([all_results_df, year_counts], ignore_index=True)

    # Reorder the DataFrame columns if needed
    all_results_df = all_results_df[['region', 'Year', 'number_of_days']]

    # You now have a DataFrame containing the region, Year, and number_of_days for all "max" files
    # Optionally, save this DataFrame to a new CSV file
    all_results_df.to_csv(f'/Users/ansonkong/Downloads/demand_prediciton_with_weather/resources/all_max_outliers_summary_{case}.csv', index=False)


    # Initialize an empty DataFrame to store all results for minimum outliers
    all_min_results_df = pd.DataFrame()

    # List all "min" files in the folder
    min_files = [f for f in os.listdir(folder) if f.endswith(f'_extreme_outliers_min_{case}.csv')]

    for file in min_files:
        # Construct the full file path
        filepath = os.path.join(folder, file)
        
        # Extract the region from the file name
        region = file.split(f'_extreme_outliers_min_{case}.csv')[0]
        
        try:
            # Attempt to read the file into a DataFrame
            df = pd.read_csv(filepath)
        except pd.errors.EmptyDataError:
            print(f"Skipping empty file: {file}")
            continue 
        
        # Group by 'Year' and count the number of observations for each year
        year_counts = df.groupby('Year').size().reset_index(name='number_of_days')
        
        # Add the region to the DataFrame
        year_counts['region'] = region
        
        # Append the result to the all_min_results_df DataFrame
        all_min_results_df = pd.concat([all_min_results_df, year_counts], ignore_index=True)

    # Reorder the DataFrame columns if needed
    all_min_results_df = all_min_results_df[['region', 'Year', 'number_of_days']]

    # You now have a DataFrame containing the region, Year, and number_of_days for all "min" files
    # Optionally, save this DataFrame to a new CSV file
    all_min_results_df.to_csv(f'/Users/ansonkong/Downloads/demand_prediciton_with_weather/resources/all_min_outliers_summary_{case}.csv', index=False)

In [136]:
import pandas as pd
import numpy as np
import os
def string_to_set(string):
    # Remove curly braces and split the string
    elements = string.replace('{', '').replace('}', '').split(',')
    # Remove any extra whitespace and single quotes from each element
    elements = [e.strip().replace("'", "") for e in elements]
    return set(elements)

for case in ['rcp45hotter']:  # Expand cases as needed
    directory = f'/Users/ansonkong/Downloads/Data for nyu work/output/future_{case}_weather'
    ci_directory = '/Users/ansonkong/Downloads/demand_prediciton_with_weather/resources/CI_results'
    output_directory = '/Users/ansonkong/Downloads/demand_prediciton_with_weather/resources/outlier_demand'
    os.makedirs(output_directory, exist_ok=True)

    rb_codes = [f'p{i}' for i in range(1, 135)]
    years = range(2020, 2100)

    for rb_code in rb_codes:
        extreme_outliers_max_list = []
        extreme_outliers_min_list = []

        ci_max_path = os.path.join(ci_directory, f'{rb_code}_CI_max.csv')
        ci_min_path = os.path.join(ci_directory, f'{rb_code}_CI_min.csv')
        
        if os.path.exists(ci_max_path) and os.path.exists(ci_min_path):
            ci_max_df = pd.read_csv(ci_max_path)
            ci_min_df = pd.read_csv(ci_min_path)
            
            for year in years:
                weather_file_path = os.path.join(directory, f'{rb_code}_WRF_Hourly_Mean_Meteorology_{year}.csv')
                demand_file_path = f'/Volumes/T7/prediction/{case}/{year}/{rb_code}_{year}_mlp_output.csv'

                if os.path.exists(weather_file_path) and os.path.exists(demand_file_path):
                    df = pd.read_csv(weather_file_path)
                    demand_df = pd.read_csv(demand_file_path)
                    
                    df['Date'] = pd.to_datetime(df['Time_UTC']).dt.date
                    demand_df['Date'] = pd.to_datetime(demand_df['Time_UTC']).dt.date
                    
                    grouped = df.groupby('Date')['T2'].agg(['max', 'min'])
                    grouped_demand = demand_df.groupby('Date')['Load'].agg(['sum'])  # Changed to sum
                    
                    # Merge to get Load sum values for dates above and below CI thresholds
                    grouped = grouped.merge(grouped_demand, left_index=True, right_index=True, how='left')
                    
                    ci_max_month = ci_max_df[(ci_max_df['rb'] == rb_code)]
                    ci_min_month = ci_min_df[(ci_min_df['rb'] == rb_code)]
                    
                    dates_above_max_ci = grouped[grouped['max'] > ci_max_month['CI_max_upper'].iloc[0]]
                    dates_below_min_ci = grouped[grouped['min'] < ci_min_month['CI_min_lower'].iloc[0]]
                    
                    for date, row in dates_above_max_ci.iterrows():
                        extreme_outliers_max_list.append({'rb': rb_code, 'Year': year, 'Date': date, 'Load_sum': row['sum']})
                    
                    for date, row in dates_below_min_ci.iterrows():
                        extreme_outliers_min_list.append({'rb': rb_code, 'Year': year, 'Date': date, 'Load_sum': row['sum']})
        
        # Convert lists to DataFrames
        extreme_outliers_max = pd.DataFrame(extreme_outliers_max_list)
        extreme_outliers_min = pd.DataFrame(extreme_outliers_min_list)
        
        # Save the results
        extreme_outliers_max.to_csv(os.path.join(output_directory, f'{rb_code}_extreme_demand_outliers_max_{case}.csv'), index=False)
        extreme_outliers_min.to_csv(os.path.join(output_directory, f'{rb_code}_extreme_demand_outliers_min_{case}.csv'), index=False)


    # Directories initialization
    directory = f'/Users/ansonkong/Downloads/Data for nyu work/averaged_{case}_weather'
    ci_directory = '/Users/ansonkong/Downloads/demand_prediciton_with_weather/resources/CI_results'
    output_directory = '/Users/ansonkong/Downloads/demand_prediciton_with_weather/resources/outlier_demand'
    os.makedirs(output_directory, exist_ok=True)

    states=['Alabama', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware', 'Florida', 'Georgia',
            'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland',
            'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey',
            'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina',
            'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming','usa']
    years = range(2020, 2100)

    extreme_outliers_max_list = []
    extreme_outliers_min_list = []

    for state in states:
        extreme_outliers_max_list = []
        extreme_outliers_min_list = []
        ci_max_path = os.path.join(ci_directory, f'{state}_CI_max.csv')
        ci_min_path = os.path.join(ci_directory, f'{state}_CI_min.csv')
        
        if os.path.exists(ci_max_path) and os.path.exists(ci_min_path):
            ci_max_df = pd.read_csv(ci_max_path)
            ci_min_df = pd.read_csv(ci_min_path)

            
            for year in years:
                all_rb_dfs = []
                weather_file_path = os.path.join(directory, f'{state}_averaged_weather_{year}.csv')
                if os.path.exists(weather_file_path):
                    if state == 'usa':
                        # If state is USA, set rb_list to include all RB codes from p1 to p134
                        rb_list = [f'p{i}' for i in range(1, 135)]
                    else:
                        state_to_ba_mapping = pd.read_csv('/Users/ansonkong/Downloads/demand_prediciton_with_weather/resources/state_to_ba_mapping.csv')
                        state_to_ba_mapping=state_to_ba_mapping[state_to_ba_mapping['state']==state]
                        
                        rb_list = string_to_set(state_to_ba_mapping.iloc[0]['reeds_ba_list'])
                    for rb_code in list(rb_list):
                        demand_file_path = f'/Volumes/T7/prediction/{case}/{year}/{rb_code}_{year}_mlp_output.csv'
                        demand_df = pd.read_csv(demand_file_path)
                    
                        demand_df['Date'] = pd.to_datetime(demand_df['Time_UTC']).dt.date
                        
                        grouped_demand = demand_df.groupby('Date')['Load'].agg(['sum']).rename(columns={'sum': rb_code})  # Rename to allow identification

                        all_rb_dfs.append(grouped_demand)  
                    if all_rb_dfs:
                        total_load_df = pd.concat(all_rb_dfs, axis=1)

                        # Sum across all numerical columns for each date to calculate 'Total_Load'
                        total_load_df['Total_Load'] = total_load_df.select_dtypes(include=[np.number]).sum(axis=1)

                        total_load_df.reset_index(inplace=True)  # Reset index to turn 'Date' back into a column
                        
                        

                    df = pd.read_csv(weather_file_path)
                    df['Date'] = pd.to_datetime(df['Time_UTC']).dt.date
                    # Find the dates with the highest and lowest T2 for each day
                    grouped = df.groupby('Date')['T2'].agg(['max', 'min'])

                    total_load_df.set_index('Date', inplace=True)

                    # Merge to get Load sum values for dates above and below CI thresholds
                    grouped = grouped.merge(total_load_df, left_index=True, right_index=True, how='left')
                        
                    # Only proceed if the month matches
                    ci_max_month = ci_max_df[ (ci_max_df['State'] == state)]
                    ci_min_month = ci_min_df[ (ci_min_df['State'] == state)]
                        
                    
                    dates_above_max_ci = grouped[grouped['max'] > ci_max_month['CI_max_upper'].iloc[0]]
                    dates_below_min_ci = grouped[grouped['min'] < ci_min_month['CI_min_lower'].iloc[0]]

 
                    for date, row in dates_above_max_ci.iterrows():
                        extreme_outliers_max_list.append({'rb': rb_code, 'Year': year, 'Date': date, 'Load_sum': row['Total_Load']})
                    
                    for date, row in dates_below_min_ci.iterrows():
                        extreme_outliers_min_list.append({'rb': rb_code, 'Year': year, 'Date': date, 'Load_sum': row['Total_Load']})

        # Convert lists to DataFrames
        extreme_outliers_max = pd.DataFrame(extreme_outliers_max_list)
        extreme_outliers_min = pd.DataFrame(extreme_outliers_min_list)

        # Save the results
        extreme_outliers_max.to_csv(os.path.join(output_directory, f'{state}_extreme_demand_outliers_max_{case}.csv'), index=False)
        extreme_outliers_min.to_csv(os.path.join(output_directory, f'{state}_extreme_demand_outliers_min_{case}.csv'), index=False)
    # Initialize an empty DataFrame to store all results
    all_results_df = pd.DataFrame()
    folder = '/Users/ansonkong/Downloads/demand_prediciton_with_weather/resources/outlier_demand'

    # List all "max" files in the folder
    max_files = [f for f in os.listdir(folder) if f.endswith(f'_extreme_demand_outliers_max_{case}.csv')]

    for file in max_files:
        # Construct the full file path
        filepath = os.path.join(folder, file)
        
        # Extract the region from the file name
        region = file.split(f'_extreme_demand_outliers_max_{case}.csv')[0]
        
        try:
            # Attempt to read the file into a DataFrame
            df = pd.read_csv(filepath)
        except pd.errors.EmptyDataError:
            print(f"Skipping empty file: {file}")
            continue 
        # Assuming 'Total_Load' is a column in 'df'
        # Group by 'Year' and calculate the average 'Total_Load' for each year
        year_average_total_load = df.groupby('Year')['Load_sum'].mean().reset_index(name='average_total_load')

        # Add the region to the DataFrame
        year_average_total_load['region'] = region

        # Append the result to the all_results_df DataFrame
        all_results_df = pd.concat([all_results_df, year_average_total_load], ignore_index=True)


    # Reorder the DataFrame columns if needed
    all_results_df = all_results_df[['region', 'Year', 'average_total_load']]

    # You now have a DataFrame containing the region, Year, and number_of_days for all "max" files
    # Optionally, save this DataFrame to a new CSV file
    all_results_df.to_csv(f'/Users/ansonkong/Downloads/demand_prediciton_with_weather/resources/all_max_outliers_demand_summary_{case}.csv', index=False)
    all_results_df= pd.DataFrame()


    # List all "min" files in the folder
    min_files = [f for f in os.listdir(folder) if f.endswith(f'_extreme_demand_outliers_min_{case}.csv')]

    for file in min_files:
        # Construct the full file path
        filepath = os.path.join(folder, file)
        
        # Extract the region from the file name
        region = file.split(f'_extreme_demand_outliers_min_{case}.csv')[0]
        
        try:
            # Attempt to read the file into a DataFrame
            df = pd.read_csv(filepath)
        except pd.errors.EmptyDataError:
            print(f"Skipping empty file: {file}")
            continue 
        print(df.groupby('Year')['Load_sum'].mean())
        # Group by 'Year' and calculate the average 'Total_Load' for each year
        year_average_total_load = df.groupby('Year')['Load_sum'].mean().reset_index(name='average_total_load')

        # Add the region to the DataFrame
        year_average_total_load['region'] = region

        # Append the result to the all_results_df DataFrame
        all_results_df = pd.concat([all_results_df, year_average_total_load], ignore_index=True)


    # Reorder the DataFrame columns if needed
    all_results_df = all_results_df[['region', 'Year', 'average_total_load']]

    # You now have a DataFrame containing the region, Year, and number_of_days for all "min" files
    # Optionally, save this DataFrame to a new CSV file
    all_results_df.to_csv(f'/Users/ansonkong/Downloads/demand_prediciton_with_weather/resources/all_min_outliers_demand_summary_{case}.csv', index=False)







/var/folders/8v/68yqll_54lzgb57b5q8_fbrc0000gn/T/ipykernel_43764/664177859.py:121: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  total_load_df['Total_Load'] = total_load_df.select_dtypes(include=[np.number]).sum(axis=1)
/var/folders/8v/68yqll_54lzgb57b5q8_fbrc0000gn/T/ipykernel_43764/664177859.py:123: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  total_load_df.reset_index(inplace=True)  # Reset index to turn 'Date' back into a column
/var/folders/8v/68yqll_54lzgb57b5q8_fbrc0000gn/T/ipykernel_43764/664177859.py:121: PerformanceWa

Year
2020    314999.315000
2021    340629.760000
2022    355003.530000
2023    326858.980000
2024    339169.537500
2025    327557.331250
2026    346047.695000
2028    335314.616667
2029    338813.906000
2034    358540.090000
2035    335762.020000
2036    335193.506667
2037    328435.104000
2039    343987.480000
2040    286994.555000
2043    311684.960000
2044    286599.110000
2048    339732.810000
2049    317497.370000
2054    341230.672857
2055    327730.330000
2058    324420.745000
2059    348882.710000
2060    317982.000000
2061    332832.640000
2062    349239.487500
2063    327347.250000
2064    329314.760000
2065    380719.360000
2069    346979.356667
2074    346424.638000
2076    334531.490000
2089    306716.220000
2094    336512.576000
2095    308344.217500
2098    339454.070000
Name: Load_sum, dtype: float64
Year
2022    51926.205000
2023    47035.966667
2025    49768.580000
2026    46571.490000
2029    51803.490000
2034    48384.840000
2036    48780.400000
2040    45251.790000

In [129]:
import pandas as pd
import numpy as np
import os
def string_to_set(string):
    # Remove curly braces and split the string
    elements = string.replace('{', '').replace('}', '').split(',')
    # Remove any extra whitespace and single quotes from each element
    elements = [e.strip().replace("'", "") for e in elements]
    return set(elements)

for case in ['rcp45cooler', 'rcp85hotter', 'rcp85cooler']:  # Expand cases as needed


    # Initialize an empty DataFrame to store all results for minimum outliers
    all_results_df = pd.DataFrame()

    # List all "min" files in the folder
    min_files = [f for f in os.listdir(folder) if f.endswith(f'_extreme_demand_outliers_min_{case}.csv')]

    for file in min_files:
        # Construct the full file path
        filepath = os.path.join(folder, file)
        
        # Extract the region from the file name
        region = file.split(f'_extreme_demand_outliers_min_{case}.csv')[0]
        
        try:
            # Attempt to read the file into a DataFrame
            df = pd.read_csv(filepath)
        except pd.errors.EmptyDataError:
            print(f"Skipping empty file: {file}")
            continue 
        # Group by 'Year' and calculate the average 'Total_Load' for each year
        year_average_total_load = df.groupby('Year')['Load_sum'].mean().reset_index(name='average_total_load')


        # Add the region to the DataFrame
        year_average_total_load['region'] = region

        # Append the result to the all_results_df DataFrame
        all_results_df = pd.concat([all_results_df, year_average_total_load], ignore_index=True)


    # Reorder the DataFrame columns if needed
    all_results_df = all_results_df[['region', 'Year', 'average_total_load']]
    print(all_results_df)
    

    # You now have a DataFrame containing the region, Year, and number_of_days for all "min" files
    # Optionally, save this DataFrame to a new CSV file
    all_results_df.to_csv(f'/Users/ansonkong/Downloads/demand_prediciton_with_weather/resources/all_min_outliers_demand_summary_{case}.csv', index=False)





        region  Year  average_total_load
0          p28  2025       139640.453333
1          p28  2027       143582.040000
2          p28  2028       147908.972500
3          p28  2030       147444.427500
4          p28  2042       145664.500000
...        ...   ...                 ...
4493  Kentucky  2089       315280.410000
4494  Kentucky  2094       334482.715714
4495  Kentucky  2095       314087.040000
4496  Kentucky  2098       337006.335000
4497  Kentucky  2099       317029.370000

[4498 rows x 3 columns]
     region  Year  average_total_load
0       p90  2022       172077.767500
1       p90  2023       162896.667500
2       p90  2025       178053.080000
3       p90  2026       166120.670000
4       p90  2029       175575.830000
...     ...   ...                 ...
2583    p12  2033        33155.450000
2584    p12  2038        34237.936667
2585    p12  2053        33971.815000
2586    p12  2069        32842.920000
2587    p12  2070        34547.086667

[2588 rows x 3 columns]
  